# Simple Specification Analysis Example

This example demonstrates a basic compliance calculations for various specs inside products. It uses the **Spec Service** to query, analyze and update the specs with the latest properties.

### Imports

Import Python Modules for executing the notebook. The requests library is used for communicating with various SystemLink Enterprise's endpoints. Scrapbook is used for running notebooks and recording data for the SystemLink Notebook Execution Service.

The SYSTEMLINK_API_KEY environment variable specifies an API key created for the user executing this notebook, which provides Role Based Access Control to the various SystemLink APIs called by this notebook. The API key will expire after 24 hours.
The SYSTEMLINK_HTTP_URI environment variable gives the base URL to the SystemLink instance executing this notebook.

In [9]:
import os
import scrapbook as sb

import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

api_key = os.getenv("SYSTEMLINK_API_KEY")
systemlink_uri = os.getenv("SYSTEMLINK_HTTP_URI")

### Input Parameters

These are the parameters that the notebook expects to be passed in by SystemLink. For notebooks designed to be triggered by pressing the 'Analyze' button in SystemLink Specs grid inside product details page, they must tag the cell with 'parameters' and at minimum specify the following in the cell metadata using the JupyterLab Property Inspector (double gear icon):

```json
{
  "papermill": {
    "parameters": {
      "spec_ids": [],
      "product_id": ""
    }
  },
  "systemlink": {
    "namespaces": [],
    "parameters": [
      {
        "display_name": "spec_ids",
        "id": "spec_ids",
        "type": "string[]"
      },
      {
        "display_name": "product_id",
        "id": "product_id",
        "type": "string"
      }
    ],
    "version": 2
  },
  "tags": ["parameters"]
}
```

For more information on how parameterization works, review the [papermill documentation](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work)."


In [10]:
spec_ids = ["1", "2", "3"]
product_id = "be108191-8b59-4c64-a8e0-8b5e02eff4a5"

### Constants

In [11]:
class ApiUrls:
    QUERY_SPECS_URL = f"{systemlink_uri}/nispec/v1/query-specs"
    UPDATE_SPECS_URL = f"{systemlink_uri}/nispec/v1/update-specs"

In [12]:
def create_post_request(url, body, headers = None):
    if not headers:
        headers = {}
    default_headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "x-ni-api-key": api_key,
    }
    headers = {**default_headers, **headers}

    response = requests.post(
        url,
        json=body,
        headers=headers,
    )
    response.raise_for_status()
    return response.json()

In [13]:
def __generate_spec_ids_filter():
    filter = ''
    for index, spec_id in enumerate(spec_ids):
        filter += f'specId == \"{spec_id}\"'
        if index < len(spec_ids) - 1:
            filter += " || "
    return filter   

def __batch_query_request(url, body, response_key):
    data = []

    response = create_post_request(url, body)
    if response is not None and response[response_key] is not None:
        data.extend(response[response_key])
    while response["continuationToken"]:
        body["continuationToken"] = response["continuationToken"]
        response = create_post_request(url, body)
        if response is not None and response[response_key] is not None:
            data.extend(response[response_key])

    return data

def query_parametric_specs(product_id):
    spec_ids_filter = __generate_spec_ids_filter()
    body = {
        "productIds": [product_id],
        "filter": f"(({spec_ids_filter}) && (type == \"PARAMETRIC\"))",
        "take": 1000
    }
    specs = __batch_query_request(ApiUrls.QUERY_SPECS_URL, body, "specs")

    return specs

In [14]:
def analyze_spec(spec):
    # TODO: Implement Spec Analysis logic here

    spec["properties"]["Spec Analyzed"] = "True"

def update_specs(specs):
    body = {
        "specs": specs
    }
    return create_post_request(ApiUrls.UPDATE_SPECS_URL, body)

def analyze_specs(specs):
    for spec in specs:
        analyze_spec(spec)

### Fetching, Analyzing and Updating Specs

In [15]:
parametric_specs = query_parametric_specs(product_id)

analyze_specs(parametric_specs)
update_specs_response = update_specs(parametric_specs)

### Store the result information so that SystemLink can access it

SystemLink uses scrapbook to store result information from each notebook execution to display to the user in the Execution Details slide-out. Here we will displaying the details of updated specs."

In [16]:
if "updatedSpecs" in update_specs_response:
    sb.glue("Updated Specs: ", update_specs_response["updatedSpecs"])
if "failedSpecs" in update_specs_response:
    sb.glue("Failed Specs: ", update_specs_response["failedSpecs"])

# Next Steps

1. Publish this notebook to SystemLink by right-clicking it in the JupyterLab File Browser with the interface as Specification Analysis.
1. Manually execute this notebook against the specs inside specs grid in product details page.
1. Go to spec details page to view the updated properties of the specs.